In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/train.csv')
df_val = pd.read_csv('/content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/val.csv')
df_test = pd.read_csv('/content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/test.csv')

In [ ]:
df_train.columns = ['id', 'input', 'label']
# df_train = df_train[['input', 'label']]
# df_train['label'].replace({'H': 1, 'M': 0}, inplace=True)
# df_train.head()

In [ ]:
df_val.columns = ['id', 'input', 'label']
df_val = df_val[['input', 'label']]
# df_val['label'].replace({'H': 1, 'M': 0}, inplace=True)

df_test.columns = ['id', 'input']
df_test = df_test[['input']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
new_df_train, _ = train_test_split(df_train, test_size=0.4, random_state=10, shuffle=True, stratify=df_train['label'])
new_df_train['label'].value_counts()

1    38721
0    38718
Name: label, dtype: int64

In [ ]:
new_df_val, _ = train_test_split(df_val, test_size=0.4, random_state=10, shuffle=True, stratify=df_val['label'])
new_df_val['label'].value_counts()

1    6453
0    6453
Name: label, dtype: int64

In [ ]:
!pip install -q transformers==4.12.2
!pip install -q datasets

     |████████████████████████████████| 3.1 MB 12.5 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
     |████████████████████████████████| 895 kB 50.8 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 3.3 MB 42.0 MB/s 
     |████████████████████████████████| 325 kB 12.6 MB/s 
     |████████████████████████████████| 134 kB 51.5 MB/s 
     |████████████████████████████████| 1.1 MB 44.1 MB/s 
     |████████████████████████████████| 212 kB 38.6 MB/s 
     |████████████████████████████████| 127 kB 51.6 MB/s 
     |████████████████████████████████| 271 kB 52.3 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 144 kB 51.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which 

In [ ]:
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

from datasets import load_dataset
from functools import partial
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from textwrap import wrap
from torch import nn
import os

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from datasets import load_dataset, load_metric
from datasets import Dataset, DatasetDict

In [ ]:
dataset_train = Dataset.from_pandas(new_df_train)
dataset_train = dataset_train.remove_columns('__index_level_0__')
dataset_val = Dataset.from_pandas(new_df_val)
dataset_val = dataset_val.remove_columns('__index_level_0__')
dataset_test = Dataset.from_pandas(df_test)

In [ ]:
d = DatasetDict()
d['train'] = dataset_train
d['validation'] = dataset_val

In [ ]:
d

DatasetDict({
    train: Dataset({
        features: ['input', 'label'],
        num_rows: 12906
    })
    validation: Dataset({
        features: ['input', 'label'],
        num_rows: 2151
    })
})

In [ ]:
MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.2",
  "type

In [ ]:
def preprocess_function(examples, tokenizer):
    result = tokenizer(examples["input"], padding='max_length', max_length=512, truncation=True)
    result["label"] = examples["label"]
    return result

In [ ]:
def preprocess_function2(examples):
    result = tokenizer(examples["input"], padding='max_length', max_length=512, truncation=True)
    return result

In [ ]:
tokenized_dataset_test = dataset_test.map(preprocess_function2, batched=True, batch_size=16)

  0%|          | 0/4034 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset_test

Dataset({
    features: ['input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 64533
})

In [ ]:
tokenized_dataset_test = tokenized_dataset_test.remove_columns('token_type_ids')
tokenized_dataset_test

Dataset({
    features: ['input', 'input_ids', 'attention_mask'],
    num_rows: 64533
})

In [ ]:
tokenized_datasets = d.map(
    partial(preprocess_function, tokenizer=tokenizer),
    batched=True,
    desc="Running tokenizer on dataset"
)

Running tokenizer on dataset:   0%|          | 0/13 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12906
    })
    validation: Dataset({
        features: ['input', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2151
    })
})

In [ ]:
len(tokenized_datasets['train']['input'][0])

90

In [ ]:
len(tokenized_datasets['train']['input'][100])

36

In [ ]:
num_labels = d['train'].to_pandas()['label'].nunique()
num_labels

2

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
class Classifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes
        self.bert = AutoModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
        for p in self.bert.parameters():
          p.requires_grad = False
  
    def forward(self, input_ids, attention_mask, labels=None):
        with torch.no_grad():
          last_hidden_state, pooled_output = self.bert(
              input_ids=input_ids,
              attention_mask=attention_mask,
              return_dict=False)
       
        logits = self.out(self.drop(pooled_output))

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.n_classes), labels.view(-1))

        output = (logits,)
        return ((loss,) + output) if loss is not None else output

In [ ]:
model = Classifier(num_labels)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.2",
  "type

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/experiment_1/results/sentiment-classifier',
    overwrite_output_dir=True,
    do_train=True,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1"
)

# training_args = TrainingArguments(
#     output_dir='/content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/experiment_1/results/sentiment-classifier',
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     fp16=True
# )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
training_args.device

device(type='cuda', index=0)

In [ ]:
device = torch.device("cuda")

In [ ]:
trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer
)

train_results = trainer.train()

The following columns in the training set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: token_type_ids, input.
***** Running training *****
  Num examples = 12906
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3228


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.686600,0.686628,0.543933,0.374761,0.596349,0.273234
2,0.683100,0.684510,0.570897,0.562352,0.574056,0.551115


The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: token_type_ids, input.
***** Running Evaluation *****
  Num examples = 2151
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/experiment_1/results/sentiment-classifier/checkpoint-1614
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in /content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/experiment_1/results/sentiment-classifier/checkpoint-1614/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/experiment_1/results/sentiment-classifier/checkpoint-1614/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: token_type_ids, input.
***** Running Evaluation *****
  Num examples = 2151
  Batch size = 8
S

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/experiment_1'
PATH = MODEL_PATH + '/model_epoch{}'.format(2)
torch.save(model.state_dict(), PATH)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
def get_preds(text): 
    inputs = tokenizer(text, return_tensors='pt', return_token_type_ids = False, max_length=512)
    # print(inputs)
    tokens = inputs.tokens()
    inputs = inputs.to(device)
    outputs = model(**inputs)[0]
    # print(outputs)
    # outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=1)
    # print(predictions)
    preds = []
    for pred in predictions.cpu().numpy():
        preds.append(pred)

    return preds

In [ ]:
tokenized_dataset_test

Dataset({
    features: ['input', 'input_ids', 'attention_mask'],
    num_rows: 64533
})

In [ ]:
df_test.head(10)

,Id,Text,Class
0,8,Деятельность по созданию новых рабочих мест и ...,1
1,12,12-18 июля 1944 года в схватке за мост на реке...,1
2,14,В целях реализации Федерального закона от 08.0...,1
3,16,Студент медик сказал что моя бессонница обосно...,0
4,17,В 1542 году король вновь заставил его внести в...,1
5,19,"В Тракторозаводском районе, на окраине Горного...",1
6,20,"Мы с Мишей расщепили наш ""батон"", а потом с по...",1
7,22,Он меня не знает.,0
8,23,"Казалось бы, чего волноваться, ведь киноэкипаж...",0
9,29,Она была в чёрной шляпе.,1


In [ ]:
torch.save(model.cpu(), '/content/drive/MyDrive/NNMethodsProject/ruatd-2022-bi/experiment_1/results/model')